<a href="https://colab.research.google.com/github/FelipeRicardo5/api-central-bank/blob/main/api_banco_central.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests pandas matplotlib seaborn numpy

In [ ]:
#!/usr/bin/env python3
"""
scrape_cotacoes.py
Script completo para a Atividade Prática 02 - Web Scraping (cotações de moedas).
Gera CSVs, gráficos, análise de correlação, previsão simples (USD) e relatório.
"""
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os
import numpy as np
from typing import Dict, List

# Configurações visuais
plt.style.use('ggplot')
sns.set(style="whitegrid")

# ----- Configurações -----
OUT_DIR = "dados_cotacoes"
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

# Período: últimos 30 dias (ajuste se desejar)
data_fim_dt = datetime.now()
data_inicio_dt = data_fim_dt - timedelta(days=30)
# Formato exigido pelo exemplo do enunciado (MM-DD-YYYY)
data_fim = data_fim_dt.strftime('%m-%d-%Y')
data_inicio = data_inicio_dt.strftime('%m-%d-%Y')

# Moedas a consultar
moedas = {
    'USD': 'Dólar Americano',
    'EUR': 'Euro',
    'GBP': 'Libra Esterlina',
    'JPY': 'Iene Japonês',
    'ARS': 'Peso Argentino'
}

# Armazenamento
dados_cotacoes: Dict[str, Dict] = {}

# ----- Função para consultar API do BCB -----
def consultar_bcb(codigo: str, inicio: str, fim: str) -> List[Dict]:
    """
    Consulta a API PTAX do Banco Central conforme enunciado.
    Retorna lista de dicionários com chaves: data, cotacaoCompra, cotacaoVenda.
    """
    url = (
        "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
        "CotacaoMoedaPeriodo(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)"
        f"?@moeda='{codigo}'&@dataInicial='{inicio}'&@dataFinalCotacao='{fim}'&$top=1000&$format=json"
    )
    try:
        resp = requests.get(url, timeout=15)
        resp.raise_for_status()
        j = resp.json()
        if 'value' in j and j['value']:
            resultados = []
            for item in j['value']:
                resultados.append({
                    'data': item.get('dataHoraCotacao'),
                    'cotacao_compra': item.get('cotacaoCompra'),
                    'cotacao_venda': item.get('cotacaoVenda')
                })
            return resultados
        else:
            return []
    except Exception as e:
        print(f"Erro ao consultar {codigo}: {e}")
        return []

# ----- Coleta -----
print("Iniciando coleta de dados...")
for codigo, nome in moedas.items():
    print(f"Consultando {nome} ({codigo})...")
    valores = consultar_bcb(codigo, data_inicio, data_fim)
    if valores:
        dados_cotacoes[codigo] = {'nome': nome, 'cotacoes': valores}
        print(f"  ✓ Obtidas {len(valores)} cotações.")
    else:
        dados_cotacoes[codigo] = {'nome': nome, 'cotacoes': []}
        print("  ✗ Nenhum dado encontrado (vazio ou erro).")

# ----- Processamento: criar DataFrames e salvar CSVs -----
print("\nProcessando e salvando CSVs...")
for codigo, bloco in dados_cotacoes.items():
    cot = bloco['cotacoes']
    if cot:
        df = pd.DataFrame(cot)
        df['data'] = pd.to_datetime(df['data'])
        df = df.sort_values('data').reset_index(drop=True)
        # salvar CSV
        arquivo_csv = os.path.join(OUT_DIR, f'cotacao_{codigo}.csv')
        df.to_csv(arquivo_csv, index=False, encoding='utf-8-sig')
        # guardar DataFrame no dicionário
        dados_cotacoes[codigo]['df'] = df
        print(f"  ✓ {arquivo_csv} ({len(df)} linhas).")
    else:
        print(f"  - {codigo}: sem dados para salvar.")

# ----- Visualização: gráfico comparativo de todas as moedas (cotacao_venda) -----
print("\nGerando gráficos...")
plt.figure(figsize=(14, 8))
any_plot = False
for codigo, bloco in dados_cotacoes.items():
    if 'df' in bloco:
        df = bloco['df']
        plt.plot(df['data'], df['cotacao_venda'], label=f"{bloco['nome']} ({codigo})")
        any_plot = True

if any_plot:
    plt.title('Cotação de Moedas em Relação ao Real (R$) - Últimos 30 dias', fontsize=16)
    plt.xlabel('Data', fontsize=12)
    plt.ylabel('Valor em R$', fontsize=12)
    plt.legend(fontsize=10)
    plt.grid(True)
    plt.tight_layout()
    arquivo_grafico = os.path.join(OUT_DIR, 'grafico_todas_moedas.png')
    plt.savefig(arquivo_grafico, dpi=300)
    plt.close()
    print(f"  ✓ Gráfico comparativo salvo em: {arquivo_grafico}")
else:
    print("  ✗ Nenhuma série disponível para o gráfico comparativo.")

# ----- Gráficos individuais (compra vs venda) -----
for codigo, bloco in dados_cotacoes.items():
    if 'df' in bloco:
        df = bloco['df']
        plt.figure(figsize=(12, 6))
        plt.plot(df['data'], df['cotacao_compra'], linestyle='-', label='Compra')
        plt.plot(df['data'], df['cotacao_venda'], linestyle='-', label='Venda')
        plt.title(f'Cotação do {bloco["nome"]} ({codigo}) em Relação ao Real (R$)', fontsize=14)
        plt.xlabel('Data')
        plt.ylabel('Valor (R$)')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        arquivo_grafico = os.path.join(OUT_DIR, f'grafico_{codigo}.png')
        plt.savefig(arquivo_grafico, dpi=300)
        plt.close()
        print(f"  ✓ Gráfico individual salvo: {arquivo_grafico}")

# ----- Variação percentual por moeda (usando cotacao_venda) -----
variacoes = []
nomes = []
for codigo, bloco in dados_cotacoes.items():
    if 'df' in bloco and len(bloco['df']) >= 2:
        df = bloco['df']
        primeiro = df.iloc[0]['cotacao_venda']
        ultimo = df.iloc[-1]['cotacao_venda']
        if primeiro is None or ultimo is None:
            variacao = 0.0
        else:
            variacao = ((ultimo - primeiro) / primeiro) * 100 if primeiro != 0 else 0.0
        variacoes.append(variacao)
        nomes.append(f"{bloco['nome']} ({codigo})")
# plot barras
if variacoes:
    plt.figure(figsize=(12, 7))
    cores = ['green' if v >= 0 else 'red' for v in variacoes]
    plt.bar(nomes, variacoes, color=cores)
    plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    for i, v in enumerate(variacoes):
        plt.text(i, v + (0.5 if v >= 0 else -1.5), f"{v:.2f}%", ha='center',
                 va='bottom' if v >= 0 else 'top', fontweight='bold')
    plt.title('Variação Percentual das Moedas em Relação ao Real (R$)')
    plt.ylabel('Variação (%)')
    plt.tight_layout()
    arquivo_grafico = os.path.join(OUT_DIR, 'grafico_variacao_percentual.png')
    plt.savefig(arquivo_grafico, dpi=300)
    plt.close()
    print(f"  ✓ Gráfico de variação percentual salvo: {arquivo_grafico}")
else:
    print("  - Não há variações suficientes para gerar o gráfico percentual.")

# ----- Análise de Correlação (exercício adicional) -----
# Montar DataFrame conjunto com cotacao_venda alinhado por data (inner join)
dfs = []
for codigo, bloco in dados_cotacoes.items():
    if 'df' in bloco:
        tmp = bloco['df'][['data', 'cotacao_venda']].copy()
        tmp = tmp.rename(columns={'cotacao_venda': f'{codigo}_venda'})
        dfs.append(tmp.set_index('data'))
if dfs:
    df_joined = pd.concat(dfs, axis=1, join='inner').dropna()
    if not df_joined.empty:
        corr = df_joined.corr()
        arquivo_corr = os.path.join(OUT_DIR, 'matriz_correlacao.csv')
        corr.to_csv(arquivo_corr, encoding='utf-8-sig')
        # plot heatmap
        plt.figure(figsize=(8, 6))
        sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
        plt.title('Matriz de Correlação (cotacao_venda)')
        plt.tight_layout()
        arquivo_heatmap = os.path.join(OUT_DIR, 'heatmap_correlacao.png')
        plt.savefig(arquivo_heatmap, dpi=300)
        plt.close()
        print(f"  ✓ Matriz de correlação salva em: {arquivo_corr} e {arquivo_heatmap}")
    else:
        print("  - DataFrames unidos resultaram em vazio (datasets não alinhados por data).")
else:
    print("  - Não há DataFrames suficientes para análise de correlação.")

# ----- Previsão simples (exercício adicional): regressão linear para USD (próximos 5 dias) -----
def previsao_linear(df: pd.DataFrame, coluna: str, dias_futuro: int = 5) -> pd.DataFrame:
    """
    Faz regressão linear simples (polinômio grau 1) sobre a série temporal
    e retorna DataFrame com previsões para os próximos `dias_futuro` dias.
    """
    df = df.copy().dropna(subset=[coluna])
    if df.empty or len(df) < 2:
        return pd.DataFrame()
    # usar índice numérico (dias desde o primeiro registro)
    df = df.sort_values('data').reset_index(drop=True)
    x = np.arange(len(df)).astype(float)
    y = df[coluna].astype(float).values
    # ajuste linear
    coef = np.polyfit(x, y, 1)  # [slope, intercept]
    # preparar previsões
    futuros_x = np.arange(len(df), len(df) + dias_futuro).astype(float)
    y_pred = np.polyval(coef, futuros_x)
    # datas previstas (adição de dias corridos)
    ultima_data = df['data'].iloc[-1]
    futuras_datas = [ultima_data + timedelta(days=i+1) for i in range(dias_futuro)]
    pred_df = pd.DataFrame({
        'data': futuras_datas,
        f'pred_{coluna}': y_pred
    })
    return pred_df

if 'USD' in dados_cotacoes and 'df' in dados_cotacoes['USD']:
    df_usd = dados_cotacoes['USD']['df'][['data', 'cotacao_venda']].copy()
    pred_usd = previsao_linear(df_usd, 'cotacao_venda', dias_futuro=5)
    if not pred_usd.empty:
        # plot histórico + previsão
        plt.figure(figsize=(12, 6))
        plt.plot(df_usd['data'], df_usd['cotacao_venda'], label='Histórico (venda)')
        plt.plot(pred_usd['data'], pred_usd['pred_cotacao_venda'], linestyle='--', label='Previsão (próx. 5 dias)')
        plt.title('Previsão simples (regressão linear) - USD (cotação venda)')
        plt.xlabel('Data')
        plt.ylabel('Valor (R$)')
        plt.legend()
        plt.tight_layout()
        arquivo_pred = os.path.join(OUT_DIR, 'previsao_usd_5dias.png')
        plt.savefig(arquivo_pred, dpi=300)
        plt.close()
        # salvar previsões em CSV
        arquivo_pred_csv = os.path.join(OUT_DIR, 'previsao_usd_5dias.csv')
        pred_usd.to_csv(arquivo_pred_csv, index=False, encoding='utf-8-sig')
        print(f"  ✓ Previsão USD gerada e salva: {arquivo_pred_csv}, gráfico: {arquivo_pred}")
    else:
        print("  - Não foi possível gerar previsão para USD (dados insuficientes).")
else:
    print("  - USD sem dados para previsão.")

# ----- Relatório (máximo 1 página) -----
print("\nGerando relatório resumo...")
relatorio_path = os.path.join(OUT_DIR, 'relatorio_resumo.txt')
with open(relatorio_path, 'w', encoding='utf-8') as f:
    f.write("Relatório Resumo - Variação de Moedas (Últimos 30 dias)\n")
    f.write("="*60 + "\n\n")
    f.write(f"Período analisado: {data_inicio_dt.strftime('%d/%m/%Y')} a {data_fim_dt.strftime('%d/%m/%Y')}\n\n")
    for codigo, bloco in dados_cotacoes.items():
        if 'df' in bloco:
            df = bloco['df']
            periodo_inicio = df['data'].min().strftime('%d/%m/%Y')
            periodo_fim = df['data'].max().strftime('%d/%m/%Y')
            minima = df['cotacao_venda'].min()
            maxima = df['cotacao_venda'].max()
            media = df['cotacao_venda'].mean()
            f.write(f"{bloco['nome']} ({codigo}):\n")
            f.write(f"  - Período disponível: {periodo_inicio} a {periodo_fim}\n")
            f.write(f"  - Cotação mínima (venda): R$ {minima:.4f}\n")
            f.write(f"  - Cotação máxima (venda): R$ {maxima:.4f}\n")
            f.write(f"  - Cotação média (venda): R$ {media:.4f}\n")
            # variação percentual
            if len(df) >= 2:
                p0 = df.iloc[0]['cotacao_venda']
                p1 = df.iloc[-1]['cotacao_venda']
                var_pct = ((p1 - p0) / p0) * 100 if p0 != 0 else 0.0
                f.write(f"  - Variação percentual no período: {var_pct:.2f}%\n")
            f.write("\n")
    # Observações / conclusões genéricas (serão personalizadas ao rodar)
    f.write("Observações gerais:\n")
    f.write("- As estatísticas acima foram calculadas com base nas cotações de venda retornadas pela API PTAX do Banco Central.\n")
    f.write("- Caso haja lacunas nas séries (dias sem cotação), as análises que exigem alinhamento por data utilizam somente as datas comuns entre as séries.\n")
    f.write("- Foi implementada uma previsão simples (regressão linear) para o USD para os próximos 5 dias — tratar como estimativa ingênua.\n")
print(f"  ✓ Relatório salvo em: {relatorio_path}")

print("\nProcesso finalizado. Verifique a pasta 'dados_cotacoes' para CSVs, gráficos e relatório.")


Iniciando coleta de dados...
Consultando Dólar Americano (USD)...
  ✓ Obtidas 110 cotações.
Consultando Euro (EUR)...
  ✓ Obtidas 110 cotações.
Consultando Libra Esterlina (GBP)...
  ✓ Obtidas 110 cotações.
Consultando Iene Japonês (JPY)...
  ✓ Obtidas 110 cotações.
Consultando Peso Argentino (ARS)...
  ✗ Nenhum dado encontrado (vazio ou erro).

Processando e salvando CSVs...
  ✓ dados_cotacoes/cotacao_USD.csv (110 linhas).
  ✓ dados_cotacoes/cotacao_EUR.csv (110 linhas).
  ✓ dados_cotacoes/cotacao_GBP.csv (110 linhas).
  ✓ dados_cotacoes/cotacao_JPY.csv (110 linhas).
  - ARS: sem dados para salvar.

Gerando gráficos...
  ✓ Gráfico comparativo salvo em: dados_cotacoes/grafico_todas_moedas.png
  ✓ Gráfico individual salvo: dados_cotacoes/grafico_USD.png
  ✓ Gráfico individual salvo: dados_cotacoes/grafico_EUR.png
  ✓ Gráfico individual salvo: dados_cotacoes/grafico_GBP.png
  ✓ Gráfico individual salvo: dados_cotacoes/grafico_JPY.png
  ✓ Gráfico de variação percentual salvo: dados_cotaco